# End to end Weekly Report Final Version
I have written this Python Program to automate a 1 Hour 30 Minute manual Excel task, reducing it to 4 Minute for me and my coworker. 

🚀 **Data Analysis & Reporting Project** 📊

### 🎯 **Objective**:
This project focuses on analyzing appointment statuses, patient details, and insurer references, with the goal of generating insightful **Appointment Turnaround Time (TAT)**, **QC TAT**, and **Conversion Rate** reports.

### 🔍 **Key Steps:**

1. **📥 Data Import & Cleaning**:
   - Imported raw data from a CSV file, focusing on key columns related to insurer references and appointment details.
   - Removed unnecessary columns and retained only the most relevant data for analysis.

2. **🔄 Data Transformation**:
   - Filtered and grouped the data based on specific criteria, such as group cases, completed cases, and pending cases.
   - Converted date columns into a proper datetime format to ensure accurate analysis.
   - Created new variables for **Appointment TAT** and **QC TAT**, categorizing them into readable formats (e.g., T+1, T+2).

3. **⏳ Pending & Non-Workable Cases**:
   - Identified pending cases and further categorized them into **Max Attempts**, **DND**, **Non-Contactable**, and more.
   - Separated out non-workable cases (e.g., due to location constraints, order cancellation, etc.) for analysis.

4. **📅 Monthly & Cumulative Reports**:
   - Generated monthly reports showing **cases received**, **non-workable data**, and **conversion rates**.
   - Used **Pivot Tables** to display **Appointment TAT** and **QC TAT** by month and added cumulative sums to track trends.

5. **📈 Conversion Calculations**:
   - Calculated **conversion rates** for **workable data**, showing how many cases were successfully processed compared to the total workable cases.

6. **📑 Final Report Generation**:
   - Consolidated all the results into an **Excel report**, containing multiple sheets for each data point (Raw Data, Cleaned Data, Converted Data, Pending Data, Non-Workable Data, etc.).

### 🛠️ **Technologies Used:**
- **Python**: Data manipulation using **Pandas** and **NumPy**.
- **Libraries**: Pandas, NumPy, datetime, openpyxl (for Excel report generation).

### 📤 **Final Output**:
An automated, structured Excel report summarizing key metrics, which includes:
- **Raw Data**
- **Cleaned Data**
- **Converted Data**
- **Pending Data**
- **Non-Workable Data**
- **Appointment TAT Report**
- **Overall Conversion Report**

This automated process streamlines data analysis and reporting, providing actionable insights at a glance. 

### 💻 **Code Implementation**:

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from datetime import datetime
from pprint import pprint

# Import Raw Data
raw_data = pd.read_csv("mis.csv", low_memory=False)

# It is for the Insurer reference
columns_to_keep = [
    "CorporateName", "RequestDate", "PatientName", "ApplicationId", "OrderID", "BookingId", 
    "PolicyNo", "Age", "Gender", "RelationShip", "EmailId", "ContactNo", "PackageName",
    "packageInvestigations", "ApptCreatedDate", "AppointmentDate", "ApptTime", 
    "SecondPreferredDate", "SecondPreferredTime", "VisitType", "ProviderName", "ProviderState", 
    "ProviderCity", "ProviderLocation", "AppointmentStatus", "ReportUploaded", "reportUrl", 
    "QcApprovedDate", "QC Approval Month", "Photo Available & Type", "ClientCity", 
    "ClientState", "ClientAddress", "ClientPincode", "AgentName", "AgentCode", "source", 
    "NumberofAttempts", "lastCallDateTime", "LastCallStatus", "ApptCreatedBySelfAllocation", 
    "ProductName", "planType", "loanId", "mphName", "PackageName", "SplitDate", 
    "ApprovalType", "AM_Name", "Escort", "PriorityAssigned", "DND"
]

# Filter this data for the Insurer reference
raw_data = raw_data[columns_to_keep]

# Define relevant columns for further processing
relevant_columns = [
    "RequestDate", "PatientName", "ApplicationId", "ReportUploaded", "AppointmentStatus", 
    "LastCallStatus", "AppointmentDate", "QcApprovedDate", "NumberofAttempts", "DND"
]

# Create a new DataFrame with relevant data
relevant_data = raw_data[relevant_columns]

In [2]:
# Filter rows where 'ApplicationId' starts with 'G' or ' G' and ends with '01'
group_case_index = relevant_data[
    relevant_data["ApplicationId"].str.strip().str.startswith("G") & 
    relevant_data["ApplicationId"].str.endswith("01")
].index

# Assign the filtered Group Cases to a new DataFrame
df = relevant_data.loc[group_case_index]

In [5]:
# Convert specified columns to datetime format
date_columns = ["RequestDate", "AppointmentDate", "QcApprovedDate"]
df[date_columns] = df[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors="coerce")

In [7]:
# Remove data from previous financial years
df = df[df["RequestDate"] > "31/03/2024"]

In [9]:
# Filter completed cases
completed_statuses = [
    "QC Approved", "QC APPROVED", "Reports Uploaded", "Reports Uploaded by DC", 
    "Appointment Attended", "QC Rejected", "Sent For Interpretation"
]
completed_cases = df[df["AppointmentStatus"].isin(completed_statuses)]

# Select relevant columns
converted = completed_cases.loc[:, [
    "RequestDate", "PatientName", "ApplicationId", "AppointmentDate", 
    "QcApprovedDate", "ReportUploaded", "AppointmentStatus"
]]

# Calculate TATs
converted["Appointment_TAT"] = (converted["AppointmentDate"] - converted["RequestDate"]).dt.days
converted["QC_TAT"] = (converted["QcApprovedDate"] - converted["AppointmentDate"]).dt.days

# Handle null values in QC_TAT
converted["QC_TAT"] = converted["QC_TAT"].fillna("Pending")

# Define TAT transformation function
def set_tat_value(value):
    if isinstance(value, (int, float)):
        if value < 0:
            return 0
        if value == 0:
            return "T0"
        if value == 1:
            return "T+1"
        if value == 2:
            return "T+2"
        if value == 3:
            return "T+3"
        if value == 4:
            return "T+4"
        if value > 4:
            return ">T+4"
    return value  # Return as-is for non-numeric values

# Apply transformation to TAT columns
converted["Appointment_TAT"] = converted["Appointment_TAT"].apply(set_tat_value)
converted["QC_TAT"] = converted["QC_TAT"].apply(set_tat_value)

In [11]:
# Filter for pending cases
pending_cases = df[~df["AppointmentStatus"].isin([
    "QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", 
    "QC Rejected", "Sent For Interpretation"
])]

# Select relevant columns
pending = pending_cases.loc[:, [
    "RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", 
    "LastCallStatus", "NumberofAttempts", "DND"
]]

# Assign 'Status' column based on conditions
pending["Status"] = None

# Assign 'Max Attempts' for cases with attempts > 30
pending.loc[pending["NumberofAttempts"] > 30, "Status"] = "Max Attempts"

# Assign 'DND' for cases marked as "Yes" in the 'DND' column where 'Status' is still NaN
pending.loc[pending["Status"].isna() & (pending["DND"] == "Yes"), "Status"] = "DND"

# Assign appointment status cases where applicable and 'Status' is still NaN
pending.loc[
    pending["Status"].isna() & 
    pending["AppointmentStatus"].isin([
        "Cancelled", "Cancelled by insurer", "Cancelled By Provider", 
        "Appointment Confirmed", "Order sent to partner"
    ]), 
    "Status"
] = pending.loc[
    pending["Status"].isna() & 
    pending["AppointmentStatus"].isin([
        "Cancelled", "Cancelled by insurer", "Cancelled By Provider", 
        "Appointment Confirmed", "Order sent to partner"
    ]), 
    "AppointmentStatus"
]

# Assign 'LastCallStatus' for remaining cases where 'Status' is still NaN
pending.loc[pending["Status"].isna(), "Status"] = pending["LastCallStatus"]

# Replace any remaining NaN in 'Status' with 'Non Contactable'
pending["Status"] = pending["Status"].fillna("Non Contactable")

In [13]:
# Update 'Status' to "Working On It" for specific cases
pending.loc[
    pending["Status"].isin([
        "Appointment Request Received", "Direct Medical", "Location Constraint", 
        "Medical Done Report Awaited", "Reminder"
    ]), 
    "Status"
] = "Working On It"

In [15]:
# Assign 'Type' as "Workable" for specific statuses
pending["Type"] = pending["Status"].apply(
    lambda status: "Workable" if status in [
        "Appointment Confirmed", "Callback", "Non Contactable", 
        "Order sent to partner", "Working On It"
    ] else "Non_Workable"
)

# Extract Non-Workable Data
Non_Workable_Data = pending[pending["Type"] == "Non_Workable"]

In [17]:
# Format "RequestDate" as month names for grouping
converted['Month'] = converted['RequestDate'].dt.strftime('%b')

# Create a pivot table to count 'ApplicationId' grouped by 'Appointment_TAT' and month
Pivot_Output = pd.pivot_table(
    converted,
    values='ApplicationId',
    index='Appointment_TAT',
    columns='Month',
    aggfunc='count',
    fill_value=0
).reset_index()

# Remove the column name for columns
Pivot_Output.columns.name = None

# Reorder the index
custom_order = [5, 1, 2, 3, 4, 0]  # Specify the desired order
Pivot_Output = Pivot_Output.reindex(custom_order).reset_index(drop=True)

# Add cumulative sum columns for each month
for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']:
    Pivot_Output[f'{month}_Cum'] = Pivot_Output[month].cumsum()

# Drop the original monthly count columns
Pivot_Output.drop(columns=['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'], inplace=True)

# Rename cumulative sum columns back to the original month names
Pivot_Output.rename(columns={f'{month}_Cum': month for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']}, inplace=True)

# Calculate percentage columns for each month
for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']:
    Pivot_Output[f'{month}_Per'] = Pivot_Output[month] / Pivot_Output.loc[5, month]

# Select final columns for the report
cols_order = [col for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'] for col in (month, f'{month}_Per')]
Appt_TAT_Report = Pivot_Output[cols_order]

In [19]:
def Calculate_Monthly_Data(input_df, output_col_name):
    
    # Format month from the RequestDate and add new column 'Month'
    input_df["Month"] = input_df["RequestDate"].dt.strftime("%b")

    # Define the month order
    month_order = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec","Jan","Feb","Mar"]

    # Count Cases for Each Month
    output_df = (input_df.groupby('Month')
                 .size()
                 .reset_index(name=output_col_name)
                 .assign(Month=lambda x: pd.Categorical(x['Month'], categories=month_order, ordered=True))
                 .sort_values('Month')
                 .reset_index(drop=True))

    # Calculate and append the grand total
    grand_total = output_df[output_col_name].sum()
    grand_total_row = pd.DataFrame({'Month': ['Grand Total'], output_col_name: [grand_total]})

    # Use pd.concat to add the grand total row to the monthly_cases DataFrame
    output_df = pd.concat([output_df, grand_total_row], ignore_index=True)
    return output_df

monthly_cases = Calculate_Monthly_Data(df, "Cases Received")
NW_monthly_cases = Calculate_Monthly_Data(Non_Workable_Data, "Non-Workable")
c_monthly_cases = Calculate_Monthly_Data(converted, "Converted Data")

C:\Users\mailt\AppData\Local\Temp\ipykernel_10012\364235957.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["Month"] = input_df["RequestDate"].dt.strftime("%b")


In [21]:
# Directly assign columns and calculate the values in one step
Overall_Conversion = monthly_cases.copy()  # Make a copy to avoid modifying the original DataFrame

# Merge the required data (Non-Workable and Converted Data) using 'Month' as the common key
Overall_Conversion = Overall_Conversion.merge(NW_monthly_cases[['Month', "Non-Workable"]], on='Month', how='left')
Overall_Conversion = Overall_Conversion.merge(c_monthly_cases[['Month', 'Converted Data']], on='Month', how='left')

# Calculate 'Workable' and 'Conversion on Workable Data' in one step
Overall_Conversion["Workable"] = Overall_Conversion["Cases Received"] - Overall_Conversion["Non-Workable"]
Overall_Conversion["Conversion on Workable Data"] = Overall_Conversion["Converted Data"] / Overall_Conversion["Workable"]

# Handle any potential division by zero or NaN values (if necessary)
Overall_Conversion["Conversion on Workable Data"].fillna(0, inplace=True)

# Output the final DataFrame
print(Overall_Conversion)

         Month  Cases Received  Non-Workable  Converted Data  Workable  \
0          Apr             251            57             191       194   
1          May             322            76             242       246   
2          Jun             348            74             268       274   
3          Jul             339            67             264       272   
4          Aug             322            52             262       270   
5          Sep             372            76             287       296   
6          Oct             362            90             231       272   
7          Nov             136            22              59       114   
8  Grand Total            2452           514            1804      1938   

   Conversion on Workable Data  
0                     0.984536  
1                     0.983740  
2                     0.978102  
3                     0.970588  
4                     0.970370  
5                     0.969595  
6                     0.8492

C:\Users\mailt\AppData\Local\Temp\ipykernel_10012\2436650987.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Overall_Conversion["Conversion on Workable Data"].fillna(0, inplace=True)


In [23]:
# Get the current date and format it as dd-mm-yyyy
from datetime import datetime
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

# Define the destination path
destination= "Report_" + current_date +".xlsx"
destination

# Create the writer variable
writer=pd.ExcelWriter(destination)
raw_data.to_excel(writer, sheet_name="Raw Data", index=False)
df.to_excel(writer, sheet_name="Cleaned Data", index=False)
converted.to_excel(writer, sheet_name="Converted Data", index=False)
pending.to_excel(writer, sheet_name="Pending Data", index=False)
Non_Workable_Data.to_excel(writer, sheet_name="Non_Workable Data", index=False)
Appt_TAT_Report.to_excel(writer, sheet_name="Appt_TAT_Report", index=False)
Overall_Conversion.to_excel(writer, sheet_name="Overall_Conversion", index=False)
writer.close()
print("Reports have been exported to the Excel Files")

Reports have been exported to the Excel Files


# Put all the codes in One Cell to save the running time of the codes

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
from datetime import datetime
from pprint import pprint

# Import Raw Data
raw_data = pd.read_csv("mis.csv", low_memory=False)

# It is for the Insurer reference
columns_to_keep = [
    "CorporateName", "RequestDate", "PatientName", "ApplicationId", "OrderID", "BookingId", 
    "PolicyNo", "Age", "Gender", "RelationShip", "EmailId", "ContactNo", "PackageName",
    "packageInvestigations", "ApptCreatedDate", "AppointmentDate", "ApptTime", 
    "SecondPreferredDate", "SecondPreferredTime", "VisitType", "ProviderName", "ProviderState", 
    "ProviderCity", "ProviderLocation", "AppointmentStatus", "ReportUploaded", "reportUrl", 
    "QcApprovedDate", "QC Approval Month", "Photo Available & Type", "ClientCity", 
    "ClientState", "ClientAddress", "ClientPincode", "AgentName", "AgentCode", "source", 
    "NumberofAttempts", "lastCallDateTime", "LastCallStatus", "ApptCreatedBySelfAllocation", 
    "ProductName", "planType", "loanId", "mphName", "PackageName", "SplitDate", 
    "ApprovalType", "AM_Name", "Escort", "PriorityAssigned", "DND"
]

# Filter this data for the Insurer reference
raw_data = raw_data[columns_to_keep]

# Define relevant columns for further processing
relevant_columns = [
    "RequestDate", "PatientName", "ApplicationId", "ReportUploaded", "AppointmentStatus", 
    "LastCallStatus", "AppointmentDate", "QcApprovedDate", "NumberofAttempts", "DND"
]

# Create a new DataFrame with relevant data
relevant_data = raw_data[relevant_columns]

# Filter rows where 'ApplicationId' starts with 'G' or ' G' and ends with '01'
group_case_index = relevant_data[
    relevant_data["ApplicationId"].str.strip().str.startswith("G") & 
    relevant_data["ApplicationId"].str.endswith("01")
].index

# Assign the filtered Group Cases to a new DataFrame
df = relevant_data.loc[group_case_index]

# Convert specified columns to datetime format
date_columns = ["RequestDate", "AppointmentDate", "QcApprovedDate"]
df[date_columns] = df[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors="coerce")

# Remove data from previous financial years
df = df[df["RequestDate"] > "31/03/2024"]

# Filter completed cases
completed_statuses = [
    "QC Approved", "QC APPROVED", "Reports Uploaded", "Reports Uploaded by DC", 
    "Appointment Attended", "QC Rejected", "Sent For Interpretation"
]
completed_cases = df[df["AppointmentStatus"].isin(completed_statuses)]

# Select relevant columns
converted = completed_cases.loc[:, [
    "RequestDate", "PatientName", "ApplicationId", "AppointmentDate", 
    "QcApprovedDate", "ReportUploaded", "AppointmentStatus"
]]

# Calculate TATs
converted["Appointment_TAT"] = (converted["AppointmentDate"] - converted["RequestDate"]).dt.days
converted["QC_TAT"] = (converted["QcApprovedDate"] - converted["AppointmentDate"]).dt.days

# Handle null values in QC_TAT
converted["QC_TAT"] = converted["QC_TAT"].fillna("Pending")

# Define TAT transformation function
def set_tat_value(value):
    if isinstance(value, (int, float)):
        if value < 0:
            return 0
        if value == 0:
            return "T0"
        if value == 1:
            return "T+1"
        if value == 2:
            return "T+2"
        if value == 3:
            return "T+3"
        if value == 4:
            return "T+4"
        if value > 4:
            return ">T+4"
    return value  # Return as-is for non-numeric values

# Apply transformation to TAT columns
converted["Appointment_TAT"] = converted["Appointment_TAT"].apply(set_tat_value)
converted["QC_TAT"] = converted["QC_TAT"].apply(set_tat_value)

# Filter for pending cases
pending_cases = df[~df["AppointmentStatus"].isin([
    "QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", 
    "QC Rejected", "Sent For Interpretation"
])]

# Select relevant columns
pending = pending_cases.loc[:, [
    "RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", 
    "LastCallStatus", "NumberofAttempts", "DND"
]]

# Assign 'Status' column based on conditions
pending["Status"] = None

# Assign 'Max Attempts' for cases with attempts > 30
pending.loc[pending["NumberofAttempts"] > 30, "Status"] = "Max Attempts"

# Assign 'DND' for cases marked as "Yes" in the 'DND' column where 'Status' is still NaN
pending.loc[pending["Status"].isna() & (pending["DND"] == "Yes"), "Status"] = "DND"

# Assign appointment status cases where applicable and 'Status' is still NaN
pending.loc[
    pending["Status"].isna() & 
    pending["AppointmentStatus"].isin([
        "Cancelled", "Cancelled by insurer", "Cancelled By Provider", 
        "Appointment Confirmed", "Order sent to partner"
    ]), 
    "Status"
] = pending.loc[
    pending["Status"].isna() & 
    pending["AppointmentStatus"].isin([
        "Cancelled", "Cancelled by insurer", "Cancelled By Provider", 
        "Appointment Confirmed", "Order sent to partner"
    ]), 
    "AppointmentStatus"
]

# Assign 'LastCallStatus' for remaining cases where 'Status' is still NaN
pending.loc[pending["Status"].isna(), "Status"] = pending["LastCallStatus"]

# Replace any remaining NaN in 'Status' with 'Non Contactable'
pending["Status"] = pending["Status"].fillna("Non Contactable")

# Update 'Status' to "Working On It" for specific cases
pending.loc[
    pending["Status"].isin([
        "Appointment Request Received", "Direct Medical", "Location Constraint", 
        "Medical Done Report Awaited", "Reminder"
    ]), 
    "Status"
] = "Working On It"

# Assign 'Type' as "Workable" for specific statuses
pending["Type"] = pending["Status"].apply(
    lambda status: "Workable" if status in [
        "Appointment Confirmed", "Callback", "Non Contactable", 
        "Order sent to partner", "Working On It"
    ] else "Non_Workable"
)

# Extract Non-Workable Data
Non_Workable_Data = pending[pending["Type"] == "Non_Workable"]

# Format "RequestDate" as month names for grouping
converted['Month'] = converted['RequestDate'].dt.strftime('%b')

# Create a pivot table to count 'ApplicationId' grouped by 'Appointment_TAT' and month
Pivot_Output = pd.pivot_table(
    converted,
    values='ApplicationId',
    index='Appointment_TAT',
    columns='Month',
    aggfunc='count',
    fill_value=0
).reset_index()

# Remove the column name for columns
Pivot_Output.columns.name = None

# Reorder the index
custom_order = [5, 1, 2, 3, 4, 0]  # Specify the desired order
Pivot_Output = Pivot_Output.reindex(custom_order).reset_index(drop=True)

# Add cumulative sum columns for each month
for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']:
    Pivot_Output[f'{month}_Cum'] = Pivot_Output[month].cumsum()

# Drop the original monthly count columns
Pivot_Output.drop(columns=['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'], inplace=True)

# Rename cumulative sum columns back to the original month names
Pivot_Output.rename(columns={f'{month}_Cum': month for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']}, inplace=True)

# Calculate percentage columns for each month
for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']:
    Pivot_Output[f'{month}_Per'] = Pivot_Output[month] / Pivot_Output.loc[5, month]

# Select final columns for the report
cols_order = [col for month in ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'] for col in (month, f'{month}_Per')]
Appt_TAT_Report = Pivot_Output[cols_order]

def Calculate_Monthly_Data(input_df, output_col_name):
    
    # Format month from the RequestDate and add new column 'Month'
    input_df["Month"] = input_df["RequestDate"].dt.strftime("%b")

    # Define the month order
    month_order = ["Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec","Jan","Feb","Mar"]

    # Count Cases for Each Month
    output_df = (input_df.groupby('Month')
                 .size()
                 .reset_index(name=output_col_name)
                 .assign(Month=lambda x: pd.Categorical(x['Month'], categories=month_order, ordered=True))
                 .sort_values('Month')
                 .reset_index(drop=True))

    # Calculate and append the grand total
    grand_total = output_df[output_col_name].sum()
    grand_total_row = pd.DataFrame({'Month': ['Grand Total'], output_col_name: [grand_total]})

    # Use pd.concat to add the grand total row to the monthly_cases DataFrame
    output_df = pd.concat([output_df, grand_total_row], ignore_index=True)
    return output_df

monthly_cases = Calculate_Monthly_Data(df, "Cases Received")
NW_monthly_cases = Calculate_Monthly_Data(Non_Workable_Data, "Non-Workable")
c_monthly_cases = Calculate_Monthly_Data(converted, "Converted Data")

# Directly assign columns and calculate the values in one step
Overall_Conversion = monthly_cases.copy()  # Make a copy to avoid modifying the original DataFrame

# Merge the required data (Non-Workable and Converted Data) using 'Month' as the common key
Overall_Conversion = Overall_Conversion.merge(NW_monthly_cases[['Month', "Non-Workable"]], on='Month', how='left')
Overall_Conversion = Overall_Conversion.merge(c_monthly_cases[['Month', 'Converted Data']], on='Month', how='left')

# Calculate 'Workable' and 'Conversion on Workable Data' in one step
Overall_Conversion["Workable"] = Overall_Conversion["Cases Received"] - Overall_Conversion["Non-Workable"]
Overall_Conversion["Conversion on Workable Data"] = Overall_Conversion["Converted Data"] / Overall_Conversion["Workable"]

# Handle any potential division by zero or NaN values (if necessary)
Overall_Conversion["Conversion on Workable Data"].fillna(0, inplace=True)

# Get the current date and format it as dd-mm-yyyy
from datetime import datetime
current_date = datetime.now().strftime('%d-%m-%Y')

# Define the destination path
destination= "Report_" + current_date +".xlsx"

# Create the writer variable
writer=pd.ExcelWriter(destination)
raw_data.to_excel(writer, sheet_name="Raw Data", index=False)
df.to_excel(writer, sheet_name="Cleaned Data", index=False)
converted.to_excel(writer, sheet_name="Converted Data", index=False)
pending.to_excel(writer, sheet_name="Pending Data", index=False)
Non_Workable_Data.to_excel(writer, sheet_name="Non_Workable Data", index=False)
Appt_TAT_Report.to_excel(writer, sheet_name="Appt_TAT_Report", index=False)
Overall_Conversion.to_excel(writer, sheet_name="Overall_Conversion", index=False)
writer.close()
print("Reports have been exported to the Excel Files")